<table width="100%" style="border:none">
  <tr>
    <td valign="top">
      <h1>GTEx RNAseq Preprocessing</h1>
      <ul>
<li>Author: Jeremy Yang
<li>Based on R code by Oleg Ursu.
<li>Clean, tidy, reshape RNAseq expression data.
<li>Compute tissue specificity index (Yanai et al., 2004).
<li>Save files for downstream SABV expression profile analytics.
<li>[About Colaboratory](https://research.google.com/colaboratory/faq.html).
      </ul>
    </td>
    <td align="right">
        <p>NIH Data Commons: Team Helium</p>
<img style="float:right" width="100" src="https://avatars2.githubusercontent.com/u/33356654?s=200&v=4" alt="HeliumDataCommons Logo" />
    </td>
  </tr>
  </table>
 

In [ ]:
import sys,os,re,time,io
import urllib.request
import numpy,scipy
import pandas
print('Python: %s\nPandas: %s'%(sys.version,pandas.__version__))

### Define input files
1. GTEx_v7_Annotations_SubjectPhenotypesDS.txt
- GTEx_v7_Annotations_SampleAttributesDS.txt
- GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct-1000.gz (1k sample)
- biomart_ENSG2NCBI.tsv

In [ ]:
data_dir_path="/home/jovyan/work/gtex/"
subj_pheno_file=data_dir_path + "GTEx_v7_Annotations_SubjectPhenotypesDS.txt"
sample_attribs_file=data_dir_path + "GTEx_v7_Annotations_SampleAttributesDS.txt"
gene_tpm_file=data_dir_path + "GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz"
biomart_file=data_dir_path + "biomart_ENSG2NCBI.tsv"

### Upload subjects datafile:
(GTEx_v7_Annotations_SubjectPhenotypesDS.txt)

In [ ]:
print('Upload GTEx Subjects datafile: ')
subjects = pandas.read_csv(subj_pheno_file, sep='\t', index_col='SUBJID')
print("dataset nrows: %d ; ncols: %d:"%(subjects.shape[0],subjects.shape[1]), file=sys.stderr)

In [ ]:
subjects.head()

In [ ]:
subjects.AGE.value_counts().sort_index()

### Keep only healthier subjects: 
(DTHHRDY = 4-point Hardy Scale Death Classification.)

In [ ]:
print("Subjects with Hardy score > 2 or NA: %d (removing)"%(subjects.query('DTHHRDY > 2').shape[0]), file=sys.stderr)
subjects = subjects.query('DTHHRDY <= 2')
print("dataset ncols: %d ; nrows: %d:"%(subjects.shape[0],subjects.shape[1]), file=sys.stderr)
subjects.DTHHRDY.value_counts(sort=True, dropna=False).sort_index()

### Upload samples datafile:
(GTEx_v7_Annotations_SampleAttributesDS.txt)

In [ ]:
print('Upload GTEx Samples datafile: ')
samples = pandas.read_csv(sample_attribs_file, sep='\t', index_col='SAMPID')
samples = samples[['SMATSSCR', 'SMTS', 'SMTSD']]
print("dataset nrows: %d ; ncols: %d:"%(samples.shape[0],samples.shape[1]), file=sys.stderr)

SUBJID is first two hyphen-delimted fields of SAMPID.

In [ ]:
samples['SUBJID'] = samples.index
samples['SUBJID'] = samples.SUBJID.str.extract('^([^-]+-[^-]+)-', expand=True)

In [ ]:
samples.head()

### MERGE samples and subjects:

In [ ]:
samples = pandas.merge(samples, subjects, how='inner', left_on='SUBJID', right_index=True)
samples.head()

### Clean & tidy cols. Remove samples with high degree of autolysis (self-digestion).

In [ ]:
samples.dropna(how='any', inplace=True)
print(samples.shape)
samples.SEX = samples.SEX.apply(lambda x: 'female' if x==2 else 'male' if x==1 else None)
samples.SEX.value_counts().sort_index()

In [ ]:
samples = samples[samples.SMATSSCR < 2]
print(samples.shape)

In [ ]:
samples.loc[(samples.SMTS.str.strip() == '') & samples.SMTSD.str.startswith("Skin -"), 'SMTS'] = 'Skin'

In [ ]:
(samples.SMTS+" : "+samples.SMTSD).value_counts().sort_index()

### READ GENE TPMs (full or demo subset)
Full file is ~56k rows, 2.6GB uncompressed.  Demo ~1k rows.

*   GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz
*   GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm_demo.gct.gz


In [ ]:
print('Upload GTEx RNAseq TPM datafile: ')
t0 = time.time()
rnaseq = pandas.read_table(open(gene_tpm_file,"rb"), compression='gzip', sep='\t', skiprows=2, nrows=1402)
print("dataset nrows: %d ; ncols: %d:"%(rnaseq.shape[0],rnaseq.shape[1]), file=sys.stderr)
print("Elapsed: %ds"%(time.time()-t0))

In [ ]:
rnaseq.info()

In [ ]:
rnaseq = rnaseq.drop(columns=['Description'])
rnaseq = rnaseq.rename(columns={'Name':'ENSG'})
rnaseq.columns

### MELT: One row per ENSG+SAMPID+TPM triplet:
Easier to handle but ~3x storage.

In [ ]:
rnaseq = rnaseq.melt(id_vars = "ENSG", var_name = "SAMPID", value_name = "TPM")

In [ ]:
rnaseq.head()

In [ ]:
rnaseq.info()

### Read and merge gene symbols.
 * /home/data/GTEx/data/biomart_ENSG2NCBI.tsv


In [ ]:
print('Upload Biomart ENSG2NCBI genes datafile: ')
genes = pandas.read_csv(biomart_file, sep='\t', usecols=[1,2,3], na_values=[''], dtype={2:str})
genes.columns = ['ENSG','NCBI','HGNC']
genes.dropna(inplace=True)

In [ ]:
genes.info()

In [ ]:
rnaseq = pandas.merge(genes, rnaseq, on='ENSG', how='inner')
rnaseq.head()

### Remove genes in pseudoautosomal regions (PAR) of chromosome Y ("ENSGR").

In [ ]:
n_ensgr = rnaseq.ENSG.str.startswith('ENSGR').sum()
print('ENSGR gene TPMs: %d (%.2f%%)'%(n_ensgr,100*n_ensgr/rnaseq.shape[0]))

In [ ]:
rnaseq = rnaseq[~rnaseq.ENSG.str.startswith('ENSGR')]

### Merge with samples:

In [ ]:
rnaseq = pandas.merge(rnaseq, samples, how="inner", left_on="SAMPID", right_index=True)

In [ ]:
rnaseq.head()



```
# This is formatted as code
```

### Remove data for gene-tissue pairs with all zero expression.

In [ ]:
max_tpm_0 = (rnaseq[['ENSG', 'SMTSD', 'TPM']].groupby(by=['ENSG','SMTSD'], as_index=True).max() == 0).rename(columns={'TPM':'max_tpm_0'})
print(max_tpm_0.max_tpm_0.value_counts())
rnaseq = pandas.merge(rnaseq, max_tpm_0, left_on=['ENSG', 'SMTSD'], right_index=True)
rnaseq = rnaseq[~rnaseq['max_tpm_0']]
rnaseq.drop(columns=['max_tpm_0'], inplace=True)

### Remove data for gene-tissue pairs not present in both sexes.

In [ ]:
sex_count = (rnaseq[['ENSG', 'SMTSD', 'SEX']].groupby(by=['ENSG','SMTSD'], as_index=True).nunique()).rename(columns={'SEX':'sex_count'})
print(sex_count.sex_count.value_counts())
rnaseq = pandas.merge(rnaseq, sex_count, left_on=['ENSG', 'SMTSD'], right_index=True)
rnaseq = rnaseq[rnaseq['sex_count'] == 2]
rnaseq.drop(columns=['sex_count'], inplace=True)

### Compute median TPM by gene+tissue:

In [ ]:
rnaseq_med = rnaseq[['ENSG', 'SMTSD', 'TPM']].groupby(by=['ENSG','SMTSD'], as_index=False).median()
print("Unique counts: genes: %d ; tissues: %d ; gene-tissue pairs: %d"%(rnaseq_med.ENSG.nunique(), rnaseq_med.SMTSD.nunique(), rnaseq_med.shape[0]))
rnaseq_med.head()

In [ ]:
rnaseq_med.to_csv('rnaseq_med.tsv', sep='\t')

In [ ]:
rnaseq_med.TPM.describe()

### LOG10(TPM+1) useful transformation.

In [ ]:
rnaseq_med['LOG_TPM'] = rnaseq_med.TPM.apply(lambda x: numpy.log10(x+1))

In [ ]:
rnaseq_med.LOG_TPM.plot(kind='box')

### Compute tissue specificity index (Yanai et al., 2004).

> ## $ \tau = \frac{\sum_{i=0}^N (1 - x_i)}{N - 1} $

> * N = number of tissues
> * x = expression profile component normalized by the maximal component value

Validate with example vector from paper.  Should be 0.95.



In [ ]:
def TAU(X):
  N = len(X)
  xmax = max(X)
  if xmax==0: return(0.0)
  tau = 0.0
  for x in X:
    tau += (1 - x/xmax)
  tau /= (N - 1)
  return(tau)
print('%.2f'%TAU([0,8,0,0,0,2,0,2,0,0,0,0]))

In [ ]:
rnaseq_tau = rnaseq_med.groupby(['ENSG']).TPM.agg(TAU)
rnaseq_tau = pandas.DataFrame(rnaseq_tau).rename(columns={'TPM':'TAU'})
rnaseq_tau.head()

In [ ]:
rnaseq_tau.plot(kind='hist')

### Assign gene-tissue rank (quantile of median) among tissues.
Low-Med-High cutoff quantiles: .25 and .75.  These ranks measure, for a given gene, relative tissue expression from the gene-tissue TPMs.

In [ ]:
def GTRanks(rnaseq, tpm_col):
  tpm_rank = pandas.Series(dtype="float", index=range(rnaseq.shape[0]))
  for i in rnaseq.index:
    ensg = rnaseq.ENSG[i]
    val_this = rnaseq[tpm_col][i]
    vals_ensg = rnaseq[tpm_col][rnaseq.ENSG==ensg]
    vals_ensg = vals_ensg.sort_values().reset_index(drop=True)
    j = vals_ensg[vals_ensg == val_this].index[0]
    tpm_rank.iloc[i] = j/vals_ensg.size 

  rnaseq[tpm_col+'_RANK'] = tpm_rank
  return(rnaseq)

In [ ]:
rnaseq_med.info()

In [ ]:
t0 = time.time()
rnaseq_level = GTRanks(rnaseq_med.copy(), 'TPM')
print("Elapsed: %ds"%(time.time()-t0))
rnaseq_level.head()

In [ ]:
rnaseq_level.TPM_RANK.plot(kind='hist')

In [ ]:
rnaseq_level.describe()

In [ ]:
rnaseq_level.to_csv('rnaseq_level.tsv', sep='\t')

In [ ]:
rnaseq_level['LEVEL'] = rnaseq_level.TPM_RANK.apply(lambda x: 'Not detected' if x==0 else 'Low' if x<.25 else 'Medium' if x<.75 else 'High')
rnaseq_level['AGE'] = 'ALL'
rnaseq_level['SEX'] = 'ALL'
rnaseq_level.head()

In [ ]:
rnaseq_level.LEVEL.value_counts().sort_index()

### Compute median TPM by gene+tissue+SEX:

In [ ]:
rnaseq_med_sex = rnaseq[['ENSG', 'SMTSD', 'SEX', 'TPM']].groupby(by=['ENSG','SMTSD','SEX'], as_index=False).median()
print(rnaseq_med_sex.shape)
rnaseq_med_sex.head()

### Combine rows into one row per gene+tissue, cols for M and F TPM.

In [ ]:
rnaseq_med_sex_f = rnaseq_med_sex.loc[rnaseq_med_sex['SEX'] == 'female']
rnaseq_med_sex_f = rnaseq_med_sex_f[['ENSG', 'SMTSD', 'TPM']].rename(columns={'TPM':'TPM_F'})
rnaseq_med_sex_m = rnaseq_med_sex.loc[rnaseq_med_sex['SEX'] == 'male']
rnaseq_med_sex_m = rnaseq_med_sex_m[['ENSG', 'SMTSD', 'TPM']].rename(columns={'TPM':'TPM_M'})
rnaseq_med_sex = pandas.merge(rnaseq_med_sex_f, rnaseq_med_sex_m, how='inner', on=['ENSG','SMTSD'])
rnaseq_med_sex.head()

In [ ]:
t0 = time.time()
rnaseq_level_f = GTRanks(rnaseq_med_sex[['ENSG','SMTSD','TPM_F']].copy(), 'TPM_F')
print("Elapsed: %ds"%(time.time()-t0))
rnaseq_level_f['LEVEL_F'] = rnaseq_level_f.TPM_F_RANK.apply(lambda x: 'Not detected' if x==0 else 'Low' if x<.25 else 'Medium' if x<.75 else 'High')
rnaseq_level_f.head()

In [ ]:
t0 = time.time()
rnaseq_level_m = GTRanks(rnaseq_med_sex[['ENSG','SMTSD','TPM_M']].copy(), 'TPM_M')
print("Elapsed: %ds"%(time.time()-t0))
rnaseq_level_m['LEVEL_M'] = rnaseq_level_m.TPM_M_RANK.apply(lambda x: 'Not detected' if x==0 else 'Low' if x<.25 else 'Medium' if x<.75 else 'High')
rnaseq_level_m.head()

In [ ]:
rnaseq_level = pandas.merge(rnaseq_level_f, rnaseq_level_m, on = ['ENSG','SMTSD'], how = 'inner')
rnaseq_level.head()

### For each gene, compute sex difference via Wilcox test:
(Wilcoxon signed-rank test, with Wilcox treatment, discarding all zero-differences.)

In [ ]:
from scipy import stats
wilcox = pandas.DataFrame({'ENSG':rnaseq_med_sex.ENSG.drop_duplicates().sort_values(), 'stat':None, 'pval':None}).reset_index(drop=True)

for i in range(wilcox.shape[0]):
  tpm_f_this = rnaseq_med_sex.TPM_F[rnaseq_level.ENSG == wilcox.ENSG[i]]
  tpm_m_this = rnaseq_med_sex.TPM_M[rnaseq_level.ENSG == wilcox.ENSG[i]]
  stat, pval = stats.wilcoxon(x=tpm_f_this, y=tpm_m_this, zero_method='wilcox')
  wilcox.stat.iloc[i] = stat
  wilcox.pval.iloc[i] = pval 
wilcox.head()

### Combine rows into one row per gene+tissue, cols for M and F TPM.

### Log fold-change is log of ratio.

In [ ]:
rnaseq_level['log2foldchange'] = ((rnaseq_level.TPM_F+1) / (rnaseq_level.TPM_M+1)).apply(lambda x: numpy.log2(max(x, 1/x)))
rnaseq_level.head()

In [ ]:
rnaseq_level.log2foldchange.plot(kind='hist', logy=True)